# Выбор оптимальных парамеров для сигнала

Подогнать параметры сигнала так, чтобы они преобрели смысл в контексте целевой функции.

Дана целевая функция $T(i): \mathbb{N} \to \{ -1, 0, 1 \}$ и сигнал $F(i, p_1, ..., p_n) \to \mathbb{R}$,
где $i \in 0,1,...,N$ - порядковый номер свечи

Возьмем множества порядковых номера интервалов, в которые желательно открывать, ничего не делать или закрывать позиции соотв.
$$T_{buy}: \{i \mid T(i) = 1\} \newline T_{nothing}: \{i \mid T(i) = 0\} \newline T_{sell}: \{i \mid T(i) = -1\}$$

Возьмем множества значений сигнала на вышеупомянутых множествах
$$F_{buy}: \{F(i, p_1, ..., p_n) \mid i \in T_{buy} \} \newline F_{nothing}: \{F(i, p_1, ..., p_n) \mid i \in T_{nothing} \} \newline F_{sell}: \{F(i, p_1, ..., p_n) \mid i \in T_{sell} \}$$


Найти такой набор параметров $p_1, ..., p_n$, чтобы минимизировать пересечения множеств:
$$\| F_{buy} \cap F_{sell} \| \newline \| F_{buy} \cap F_{nothing} \| \newline \| F_{sell} \cap F_{nothing} \|$$

В качестве метрики возьмем
$$ Q = \| F_{buy} \cap F_{sell} \| + \| F_{buy} \cap F_{nothing} \| + \| F_{sell} \cap F_{nothing} \|$$

PS. Метрика - говно. Может случиться так, что пересечений почти нет, но на самом деле это случайность. Например, сигналы на покупку - [1, 2, 3], сигналы на продажу - [1.01, 2.01, 3.01]. Пересечений нет, но на самом деле использовать такой сигнал для обучения нет смысла (он просто подогнан под исторические данные)

Вариант - получать оболочку множеств и смотреть площадь пересечения оболочек. Еще вариант - предположить, что множества представляют собой сэмплы случайных величин. Тогда можно оценивать расстояние между ожиданиями этих величин с учетом стандартного отклонения.


## Забираем исторические данные

И немного сглаживаем.

In [ ]:
import datetime

import plotly.graph_objects as go
import pandas as pd

from acquisition import fetch_candle_data
from processing.plot_utils import make_candlestick

first_date = datetime.date(year=2019, month=1, day=1)
last_date = datetime.date(year=2021, month=7, day=1)

candle_data = fetch_candle_data(
    ticker='NVDA',
    first_date=first_date,
    last_date=last_date)

raw_close: 'pd.Series[float]' = candle_data.c
smoothed_close: 'pd.Series[float]' = raw_close.rolling(window=3, win_type='gaussian').mean(std=0.9)

candles_fig = go.FigureWidget(make_candlestick(candle_data[30:200]))
close_fig = go.FigureWidget()
close_fig.add_scatter(x=candle_data.t[30:200], y=raw_close[30:200], name='raw close')
close_fig.add_scatter(x=candle_data.t[30:200], y=smoothed_close[30:200], name='smoothed close')

candles_fig.show()
close_fig.show()


## Вычисляем целевую переменную

In [ ]:
import typing
import functools

import pandas as pd
import plotly.graph_objects as go

from processing.target import eval_target


def target_strategy_1(
        close: 'pd.Series[float]',
        buy_threshold: float,
        sell_threshold: float,
        do_nothing_threshold: float) -> float:
    current_price = typing.cast(float, close.values[0])

    buy_freq = 0
    sell_freq = 0

    for idx in range(1, len(close)):
        ret = (
            typing.cast(
                float,
                close.values[idx]) - current_price) / current_price
        if ret > buy_threshold:
            buy_freq += 1
        if ret < -sell_threshold:
            sell_freq += 1

    signal_strength = abs(buy_freq - sell_freq) / (len(close) - 1)
    if signal_strength < do_nothing_threshold:
        return 0.0

    if buy_freq > sell_freq:
        return buy_freq / (len(close) - 1)
    if sell_freq > buy_freq:
        return -sell_freq / (len(close) - 1)

    return 0.0


window = 7
buy_threshold = 0.0064
sell_threshold = 0.0032
do_nothing_threshold = 0.3

strategy = functools.partial(
    target_strategy_1,
    buy_threshold=buy_threshold,
    sell_threshold=sell_threshold,
    do_nothing_threshold=do_nothing_threshold)

target = eval_target(smoothed_close, window, strategy).actions
target[target > 0] = 1.0
target[target < 0] = -1.0

target_fig = go.FigureWidget()
target_fig.add_scatter(x=candle_data.t[30:200], y=target[30:200], name='target')
target_fig.show()

## Подбираем параметры для MACD

In [ ]:
import numpy as np

from plotly.subplots import make_subplots

from ta.trend

min_q = 100000000000
min_window_slow = 0
min_window_fast = 0

for window_fast in range(2, 50):
    for window_slow in range(window_fast + 1, 100):
        macd = MACD(
            close=candle_data.c,
            window_slow=window_slow,
            window_fast=window_fast).macd_diff()

        index = macd.index.intersection(target.index)
        target_tmp = target.loc[index]
        macd = macd.loc[index]
        macd = macd / macd.abs().max()

        nothing = macd[target_tmp == 0]
        buy = macd[target_tmp > 0]
        sell = macd[target_tmp < 0]

        buy_std = np.std(buy)
        sell_std = np.std(sell)
        nothing_std = np.std(nothing)
        
        q = -nothing_std
        if q < min_q:
            min_q = q
            min_window_fast = window_fast
            min_window_slow = window_slow

print(f'Min q: {min_q}; window_fast: {min_window_fast}; window_slow: {min_window_slow}')





In [ ]:
macd = MACD(
    close=candle_data.c,
    window_slow=min_window_slow,
    window_fast=min_window_fast).macd_diff()

index = macd.index.intersection(target.index)
target_tmp = target.loc[index]
macd = macd.loc[index]

macd_buy = macd[target_tmp > 0]
macd_sell = macd[target_tmp < 0]
macd_nothing = macd[target_tmp==0]

macd_fig = make_subplots(specs=[[{"secondary_y": True}]])

macd_fig.add_scatter(
    x=candle_data.t[30:200], y=macd[30:200], name='macd', secondary_y=False)
macd_fig.add_scatter(x=candle_data.t[30:200],
                     y=target[30:200],
                     name='target',
                     secondary_y=False)
macd_fig.add_scatter(x=candle_data.t[30:200],
                     y=smoothed_close[30:200],
                     name='close',
                     secondary_y=True)
macd_fig.show()

sets_fig = go.FigureWidget()
sets_fig.add_histogram(x=macd_buy, name='buy', histnorm='probability density')
sets_fig.add_histogram(x=macd_sell, name='sell', histnorm='probability density')
sets_fig.add_histogram(x=macd_nothing, name='nothing', histnorm='probability density')
sets_fig.show()

In [ ]:
volatility = (candle_data.v.rolling(window=8).mean() - candle_data.v.rolling(window=13).mean()) / candle_data.v.rolling(window=5).mean()

index = volatility.index.intersection(target.index)
target = target.loc[index]
volatility = volatility.loc[index]

v_buy = volatility[target > 0]
v_sell = volatility[target < 0]
v_nothing = volatility[target==0]

volatility_fig = make_subplots(specs=[[{"secondary_y": True}]])

volatility_fig.add_scatter(
    x=candle_data.t[30:200], y=volatility[30:200], name='volatility', secondary_y=False)
volatility_fig.add_scatter(x=candle_data.t[30:200],
                     y=target[30:200],
                     name='target',
                     secondary_y=False)
volatility_fig.add_scatter(x=candle_data.t[30:200],
                     y=smoothed_close[30:200],
                     name='close',
                     secondary_y=True)
volatility_fig.show()

v_sets_fig = go.FigureWidget()
v_sets_fig.add_histogram(x=v_buy, name='buy', histnorm='probability density')
v_sets_fig.add_histogram(x=v_sell, name='sell', histnorm='probability density')
v_sets_fig.add_histogram(x=v_nothing, name='nothing', histnorm='probability density')
v_sets_fig.show()